# students-to-course-assigner

## Imports

In [ ]:
import os

import pandas as pd
from ortools.sat.python import cp_model
import math
from dataclasses import dataclass, field
from typing import List
from collections import defaultdict

## Data structures

In [ ]:
@dataclass
class Student:
    id_: int
    first_name: str
    last_name: str
    class_: str
    desired_courses: List[str]
    assigned_course: str = None

In [ ]:
@dataclass
class Course:
    id_: str
    min_students: int
    max_students: int
    nb_wishes: int = 0
    assigned_students: List[int] = field(default_factory=lambda : [])
    def assign_if_possible(self, student):
        if len(self.assigned_students) < self.max_students:
            self.assigned_students += [student]
            student.assigned_course = self

## Read data

In [ ]:
def read_data(excel_file_name, module_type):
    # read data from excel
    if module_type not in ["A", "B"]:
        raise ValueError(f"module_type must be A or B, but is {module_type}")
    excel_path = os.path.join("data", excel_file_name)
    df_students = pd.read_excel(excel_path, sheet_name="Rohdaten").fillna("").reset_index()
    df_courses = pd.read_excel(excel_path, sheet_name="Module")
    # collect students
    student_per_student_id = {}
    for e in df_students.to_dict(orient="records"):
        cs = []
        for i in range(1, 5):
            c = e[f"Def {module_type} - {i}. Prio"].replace(" ", "")
            cs += [c]
        if not all(not c for c in cs):
            if any(not c for c in cs):
                print(f"student has not all courses not defined, but any, is ignored: {e['index']}")
            else:
                id_ = e["index"]
                student_per_student_id[id_] = Student(id_, e["Vorname"], e["Name"], e["Klasse"], cs)
        else:
            if module_type != "A" or (e["Klasse"].startswith("27") and e["Klasse"][3] not in ["s", "t"]):
                print(f"student has all courses not defined, is ignored: {e['index']}")
    print(f"# students with valid data: {len(student_per_student_id)}")
    # collect courses
    course_per_course_id = {}
    for e in df_courses.to_dict(orient="records"):
        id_ = e["Module"]
        if id_.startswith(module_type):
            course_per_course_id[id_] = Course(id_, e["Min"], e["Max"])
    print(f"# courses before harmonization: {len(course_per_course_id)}")
    # count wishes
    nb_wishes_per_course_id = defaultdict(int)
    for s in student_per_student_id.values():
        for c_id in s.desired_courses:
            nb_wishes_per_course_id[c_id] += 1
    for c in course_per_course_id.values():
        c.nb_wishes = nb_wishes_per_course_id[c.id_]
    # remove courses with not enough wishes
    courses_to_remove = []
    for c in course_per_course_id.values():
        if c.nb_wishes < c.min_students:
            courses_to_remove += [c]
    for c in courses_to_remove:
        print(f"course {c} has not enough wishes, is removed")
        for s in student_per_student_id.values():
            if c.id_ in s.desired_courses:
                s.desired_courses.remove(c.id_)
        del course_per_course_id[c.id_]
    print(f"# courses after harmonization: {len(course_per_course_id)}")
    return student_per_student_id, course_per_course_id

## Assignment with or-tools

In [ ]:
def get_cost(student, course_id):
    if course_id in student.desired_courses:
        return student.desired_courses.index(course_id)
    else:
        return 1000

In [ ]:
def assign_students_to_courses(student_per_student_id, course_per_course_id, excel_name_out):
    # init assignment model
    model = cp_model.CpModel()
    # define boolean variables
    x = {}
    for student_ind, _ in enumerate(student_per_student_id.values()):
        for course_ind, _ in enumerate(course_per_course_id.values()):
            x[student_ind, course_ind] = model.NewBoolVar(f"x[{student_ind},{course_ind}]")
    # each student is assigned to exactly one course.
    for student_ind, _ in enumerate(student_per_student_id.values()):
        model.AddExactlyOne(x[student_ind, course_ind] for (course_ind, course) in enumerate(course_per_course_id.values()))
    # each course takes at most n students
    for course_ind, course in enumerate(course_per_course_id.values()):
        model.Add(sum([x[student_ind, course_ind] for (student_ind, _) in enumerate(student_per_student_id.values())]) >= course.min_students)
        model.Add(sum([x[student_ind, course_ind] for (student_ind, _) in enumerate(student_per_student_id.values())]) <= course.max_students)
    # objective function
    objective_terms = []
    for student_ind, student in enumerate(student_per_student_id.values()):
        for course_ind, course in enumerate(course_per_course_id.values()):
            objective_terms += [get_cost(student, course.id_) * x[student_ind, course_ind]]
    model.Minimize(sum(objective_terms))
    # solve
    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    print(solver.SolutionInfo())
    print(solver.ResponseStats())
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print(f"total cost = {solver.ObjectiveValue()}\n")
        # collect results from assignment
        for student_ind, student in enumerate(student_per_student_id.values()):
            for course_ind, course in enumerate(course_per_course_id.values()):
                if solver.BooleanValue(x[student_ind, course_ind]):
                    student.assigned_course = course.id_
                    course.assigned_students += [student]
        # statistics about students without course
        students_without_course = [s for s in student_per_student_id.values() if s.assigned_course is None]
        print(f"# students without course: {len(students_without_course)}")
        # statistics about students without course
        students_with_not_desired_course = [s for s in student_per_student_id.values() if not s.assigned_course in s.desired_courses]
        print(f"# students not with desired course: {len(students_with_not_desired_course)}")
        # statistics about courses with number of students not in min-max-range
        for course in course_per_course_id.values():
            nb_students = len(course.assigned_students)
            if nb_students < course.min_students:
                print(f"not enough students: {course, nb_students}")
            if nb_students > course.max_students:
                print(f"too much students: {course, nb_students}")
        # statistics about courses
        course_statistics = []
        for c in course_per_course_id.values():
            course_statistics += [[c.id_, c.min_students, c.max_students, c.nb_wishes, len(c.assigned_students)]]
        df_course_statistics = pd.DataFrame(course_statistics, columns=["course_id", "min_students", "max_students", "nb_wishes", "nb_students_assigned"])
        # statistics about students
        nb_students_per_priority = defaultdict(int)  
        students_infos = []
        for s in student_per_student_id.values():
            if s.assigned_course in s.desired_courses:
                nb_students_per_priority[s.desired_courses.index(s.assigned_course)] += 1
                ind = s.desired_courses.index(s.assigned_course)
            else:
                nb_students_per_priority["not_desired"] += 1
                ind = "None"
            students_infos += [[s.id_, s.first_name, s.last_name, s.class_, ",".join(s.desired_courses), s.assigned_course, ind]]
        df_students = pd.DataFrame(students_infos, columns=["student_id", "first_name", "last_name", "class", "desired_courses", "assigned_course", "priority"])        
        df_student_statistics = pd.DataFrame(sorted(nb_students_per_priority.items(), key=lambda x: str(x[0])), columns=["priority", "nb_students"])
        if not os.path.exists("out"):
            # If it doesn't exist, create the folder
            os.makedirs("out")
        with pd.ExcelWriter(os.path.join("out", excel_name_out)) as writer:
            df_course_statistics.to_excel(writer, sheet_name='course_statistics', index=False)
            df_student_statistics.to_excel(writer, sheet_name='student_statistics', index=False)
            df_students.to_excel(writer, sheet_name='students', index=False)
        return df_course_statistics, df_student_statistics, df_students
    else:
        print("No solution found.")
        return None, None, None

## Execution

In [ ]:
MODULE_TYPE = "B"

In [ ]:
student_per_student_id, course_per_course_id = read_data("InputNeufeld_Dez2023_erw.xlsx", MODULE_TYPE)

In [ ]:
df_course_statistics, df_student_statistics, df_students = assign_students_to_courses(student_per_student_id, course_per_course_id, f"result_{MODULE_TYPE}_erw.xlsx")

In [ ]:
df_course_statistics

In [ ]:
df_student_statistics

In [ ]:
df_students